## Google sheets api usage without Service Account
ref. [Google Sheets API Quickstart](https://developers.google.com/sheets/api/quickstart/python)

### Steps
1. Create a project in Google Cloud.
2. Enable Google Sheets API.
3. Configure OAuth:
    1. Go to OAuth consent screen.
    2. Select user type "External" (since "Internal" is disabled for normal Google users).
    3. Provide a random website address in app domain settings (if a non-existent website doesn't work, provide any real website, e.g., `https://www.randomwebsite.com`).
    4. Enter the same in the authorized domain, e.g., `randomwebsite.com`.
    5. Enter your email.
    6. Add a test user with your email.
4. Create credentials and save the JSON as `credentials.json`.
5. Install Python libraries and run.


In [44]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [45]:

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"] # remove.readonly if you want to write to the sheet

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1MDXKcN6B6xDFkSx4iNXYmXPJQDh_TGblSTPXi7Y8uIA"
SAMPLE_RANGE_NAME = "Sheet1!A2:C6"


## Auth using Google library

In [ ]:

# def main():
"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=879966407845-30rpk9p1l7fulmjit3c9v4mgh0fq99s8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51825%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=3yKmTMuCH2wBw9k2FG2BqZTlEIpkFr&access_type=offline


## Use the api

In [42]:

try:
  service = build("sheets", "v4", credentials=creds)

  # Call the Sheets API
  sheet = service.spreadsheets()
  result = (
      sheet.values()
      .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
      .execute()
  )
  values = result.get("values", [])

  if not values:
      print("No data found.")
    #   return

  for row in values:
    # Print columns A and E, which correspond to indices 0 and 4.
    print(f"{row}")
except HttpError as err:
  print(err)


[]
['frame', 'timestamp (ns)', 'tstamp diff (ns)']
['109', '1690945602775932000']
['110', '1690945602875924200', '99989760']
['111', '1690945602975930600', '100010240']


## Auth using manual process

In [46]:
import json
with open("credentials.json", encoding='utf-8') as f:
    credentials_json = json.load(f)

In [47]:
# generate authorization url
import requests
import socket

def find_unused_port():
    # Create a socket and bind it to an available port
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('localhost', 0))  # Bind to localhost with port 0
        _, port = s.getsockname()  # Get the port assigned by the OS
    return port

# Example usage
unused_port = find_unused_port()
# print(f"Unused port: {unused_port}")

response_type = 'code'
client_id = credentials_json['installed']['client_id']
redirect_uri = credentials_json['installed']['redirect_uris'][0] + f':{unused_port}'
scope = SCOPES[0]
state='Yvz6jkpKxsesWxCOmFZ5ORY7RqyiHO' # any random string of this length
access_type='offline'

auth_uri = credentials_json['installed']['auth_uri']

params = {
    "response_type": response_type,
    "client_id": client_id,
    "redirect_uri": redirect_uri,
    "scope": scope,
    "state": state,
    "access_type": access_type
    }

authorization_url = requests.Request("GET", auth_uri, params=params).prepare().url
print(authorization_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=879966407845-30rpk9p1l7fulmjit3c9v4mgh0fq99s8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55407&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=Yvz6jkpKxsesWxCOmFZ5ORY7RqyiHO&access_type=offline


complete user consent from above url, then copy the auth code from redirected url params

In [50]:
authorization_code = '4/0AanRRrtN0p9D7TTgtMjyCZVxYd03rv51_UQlw9MVpPk2EcksYpbPOM_Bj-JMymX1rFueGQ'

In [51]:
token_data = {
    "grant_type": "authorization_code",
    "code": authorization_code,
    'client_id': client_id,
    'client_secret': credentials_json['installed']['client_secret'],
    'redirect_uri': redirect_uri
}
token_response = requests.post("https://oauth2.googleapis.com/token", data=token_data)

if token_response.status_code != 200:
    print('Error in getting access token')
    print(token_response.json())
else:
    token_json = token_response.json()
    token_json.update(credentials_json['installed'])
    creds = Credentials.from_authorized_user_info(token_json, SCOPES)
    service = build("sheets", "v4", credentials=creds)

    # now use this service object to access the sheets api


In [52]:
# Call the Sheets API
sheet = service.spreadsheets()
result = (
    sheet.values()
    .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
    .execute()
)
values = result.get("values", [])

if not values:
    print("No data found.")
#   return

for row in values:
    # Print columns A and E, which correspond to indices 0 and 4.
    print(f"{row}")

[]
['frame', 'timestamp (ns)', 'tstamp diff (ns)']
['109', '1690945602775932000']
['110', '1690945602875924200', '99989760']
['111', '1690945602975930600', '100010240']
